In [2]:
from instancePDSVRP import instancePDSVRP
import copy
import time
import heuristic as h
import utilities as u

Without local search (also initial_solution method has to be modified)

In [3]:
def initial_solution_construction_no_LS(instance, w1, w2, w3, w4, w5, gamma):
    A = [c for c in range (1, instance.N)]
    solution = h.recreate(instance, [[[[] for _ in range(instance.h)], [[] for _ in range(instance.D)]],A], w1, w2, w3, w4, w5, gamma)
    return solution

In [4]:
def SISSRs_no_LS(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = initial_solution_construction_no_LS(instance, w1, w2, w3, w4, w5, gamma)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = h.ruin_and_recreate(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if u.cost(instance, s) < u.cost(instance, s_curr)*(1+delta):
            s_curr = s
            if u.cost(instance, s_curr) < u.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = h.perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1
    
    return s_best

Without threshold acceptence

In [5]:
def SISSRs_no_TA(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = h.initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = h.ruin_and_recreate(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if u.cost(instance, s) < u.cost(instance, s_curr):
            s_curr = h.local_search(instance, s, n_nearest)
            if u.cost(instance, s_curr) < u.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = h.perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        iteration_counter+=1
    
    return s_best

Without perturbation

In [6]:
def SISSRs_no_pert(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = h.initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    
    while (iteration_counter < iter_max):
        s = h.ruin_and_recreate(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if u.cost(instance, s) < u.cost(instance, s_curr) * (1 + delta):
            s_curr = h.local_search(instance, s, n_nearest)
            if u.cost(instance, s_curr) < u.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        delta = delta * epsilon
        iteration_counter += 1
    
    return s_best


Without sweep removal (also ruin_and_recreate method has to be modifed)

In [7]:
def ruin_and_recreate_no_sweep(instance, solution, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma):
   
    solution = h.random_drone_customer_removal(solution, sigma)

    solution = h.string_removal(instance, solution, c_average_removed, L_max)

    solution = h.recreate(instance, solution, w1, w2, w3, w4, w5, gamma)

    return solution

In [8]:
def SISSRs_no_sweep(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    
    s_0 = h.initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = ruin_and_recreate_no_sweep(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if u.cost(instance, s) < u.cost(instance, s_curr)*(1+delta):
            s_curr = h.local_search(instance, s, n_nearest)
            if u.cost(instance, s_curr) < u.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = h.perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1
    
    return s_best

Without random removal (also ruin_and_recreate method has to be modifed)

In [9]:
def ruin_and_recreate_no_rand(instance, solution, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma):
   
    solution = h.sweep_removal_operator(instance, solution, sigma)

    solution = h.string_removal(instance, solution, c_average_removed, L_max)

    solution = h.recreate(instance, solution, w1, w2, w3, w4, w5, gamma)

    return solution


In [10]:
def SISSRs_no_rand(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = h.initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = ruin_and_recreate_no_rand(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if u.cost(instance, s) < u.cost(instance, s_curr)*(1+delta):
            s_curr = h.local_search(instance, s, n_nearest)
            if u.cost(instance, s_curr) < u.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = h.perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1
    
    return s_best

Tests

In [11]:
def test_variants_on_instances(original_method, variants, instances,  sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    results = []
            
    for instance_name in instances:
        print("Original method for instance " + instance_name)
        instance = instancePDSVRP("instances/" + instance_name + ".txt") 
        
        start_time = time.time()
        sol_original = original_method(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb)
        end_time = time.time()
        cost_original = u.cost(instance, sol_original)
        elapsed_time_original = end_time - start_time

        for variant in variants:
            variant_name = variant.__name__
            print(variant_name)
                
            start_time = time.time()
            sol_variant = variant(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb)
            end_time = time.time()
            cost_variant =u.cost(instance, sol_variant)
            elapsed_time_variant = end_time - start_time
                
            cost_difference = ((cost_variant - cost_original) / cost_original) * 100
                
            results.append({
                "method": original_method.__name__,
                "variant": variant_name,
                "instance": instance_name,
                "cost_original": cost_original,
                "cost_variant": cost_variant,
                "cost_difference_percentage": cost_difference,
                "time_original": elapsed_time_original,
                "time_variant": elapsed_time_variant,
                "iter_imp" : iter_imp,
                "iter_max": iter_max
            })
    
    return results

In [ ]:
method = h.SISSRs
variants = [SISSRs_no_LS, SISSRs_no_TA, SISSRs_no_pert, SISSRs_no_rand, SISSRs_no_sweep]
instances = []

for customers_position in ["c", "r", "rc"]:
    for depot_position in ["c", "e", "r"]:
        instances.append("30-" + customers_position + "-1-" + depot_position)
        
#instances = ["small_instances/10-c-1-c"]

results = test_variants_on_instances(method, variants, instances, 0.3, 4.5, 4.5, 5,1,1,2,2, 0.1, 20, 0.1, 0.999975, 500, 5000, 3, 3, 9)

u.save_results_to_csv(results, "results/impact_of_components_results.csv")